## Stage 1 Binary Classification ResNet50

In [1]:
!pip install kaggle --quiet

import os
os.environ['KAGGLE_CONFIG_DIR'] = "~/.kaggle"  # Update this path if needed

# I had to do this because it wasn't seeing my kaggle.json
os.environ['KAGGLE_USERNAME'] = 'oscarjp'
os.environ['KAGGLE_KEY'] = 'd371e4dda7ac9f5c9825e5075716d987'

!kaggle competitions download -c rsna-pneumonia-detection-challenge

import zipfile
with zipfile.ZipFile('rsna-pneumonia-detection-challenge.zip', 'r') as zip_ref:
    zip_ref.extractall('rsna_data')

import os
os.listdir('rsna_data')

['stage_2_detailed_class_info.csv',
 'stage_2_train_images',
 'stage_2_sample_submission.csv',
 'stage_2_train_labels.csv',
 'GCP Credits Request Link - RSNA.txt',
 'stage_2_test_images']

In [1]:
# !pip install pydicom
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


2025-05-08 23:32:14.530849: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746761534.584314     571 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746761534.600307     571 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746761534.720524     571 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746761534.720570     571 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746761534.720571     571 computation_placer.cc:177] computation placer alr

Num GPUs Available:  1


In [3]:
import psutil
print(f"Available memory: {psutil.virtual_memory().available / 1e9:.2f} GB")


Available memory: 14.77 GB


In [3]:
import os
import numpy as np
import pandas as pd
import pydicom
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from tensorflow.keras.utils import Sequence
import cv2

In [4]:
# Utilizing this as a source to batch train
# https://www.tensorflow.org/api_docs/python/tf/keras/utils/PyDataset

class DICOMTrainBatch(tf.keras.utils.Sequence):
    def __init__(self, df, data_dir, batch_size=32, img_size=(224, 224), shuffle=True):
        self.df = df
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.indices = np.arange(len(df))
        self.on_epoch_end()
    
    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch_df = self.df.iloc[batch_indices]

        batch_images = []
        batch_labels = []

        for _, row in batch_df.iterrows():
            image_path = os.path.join(self.data_dir, f'{row["patientId"]}.dcm')
            image = load_dicom_image(image_path)
            batch_images.append(image)
            batch_labels.append(row["Target"])

        batch_images = np.array(batch_images) / 255.0
        batch_labels = np.array(batch_labels)

        return batch_images, batch_labels

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)


In [29]:
data_dir = './rsna_data/stage_2_train_images'
labels_df = pd.read_csv('./rsna_data/stage_2_train_labels.csv')


In [7]:

def load_dicom_image(image_path):
    dicom_data = pydicom.dcmread(image_path)

    image = dicom_data.pixel_array

    if len(image.shape) == 2:
        image = np.stack([image] * 3, axis=-1)  # Convert grayscale to RGB

    image = cv2.resize(image, (224, 224))

    image = image / 255.0

    image = (image * 255).astype(np.uint8)

    return image

# images = []
# labels = []
# for idx, row in labels_df.iterrows():
#     image_path = os.path.join(data_dir, f'{row["patientId"]}.dcm')
#     images.append(load_dicom_image(image_path))
#     labels.append(row["Target"])

# images = np.array(images)
# labels = np.array(labels)




In [6]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(labels_df, test_size=0.2, random_state=42)


In [ ]:
train_generator = DICOMTrainBatch(train_df, data_dir, batch_size=32)
validation_generator = DICOMTrainBatch(val_df, data_dir, batch_size=32, shuffle=False)


In [14]:

# X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# '''train_datagen = ImageDataGenerator(
#     rescale=1./255,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True
# )'''
# train_datagen = ImageDataGenerator(rescale=1./255)

# val_datagen = ImageDataGenerator(rescale=1./255)

# train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
# validation_generator = val_datagen.flow(X_val, y_val, batch_size=32)


In [7]:

from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.layers import Input
inputs = Input(shape=(224, 224, 3))


x = inputs
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

base_model.trainable = False
x = base_model(x, training=False)

# Custom classification head
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
outputs = Dense(1, activation='sigmoid')(x)

model = Model(inputs, outputs)


I0000 00:00:1746842975.916008     555 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5564 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:07:00.0, compute capability: 8.6


In [8]:

from tensorflow.keras.optimizers import Adam
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
	metrics=['accuracy']
)

In [ ]:
from tensorflow.keras import callbacks
# --- TRAINING ---
early_stop = callbacks.EarlyStopping(patience=5, restore_best_weights=True, min_delta=0.001, monitor='accuracy', mode='max')
reduce_lr = callbacks.ReduceLROnPlateau(patience=3, factor=0.5, verbose=1)
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=50,
    callbacks=[early_stop]
)

/home/ojp/projects/tf217/tf217/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50


I0000 00:00:1746625703.171531    9302 service.cc:152] XLA service 0x7f52d404f910 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1746625703.171571    9302 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2025-05-07 09:48:23.359736: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1746625704.171526    9302 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-05-07 09:48:25.431931: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_5837', 204 bytes spill stores, 204 bytes spill loads

2025-05-07 09:48:25.609538: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_907

  2/756 ━━━━━━━━━━━━━━━━━━━━ 40s 54ms/step - accuracy: 0.5469 - loss: 0.8321   

I0000 00:00:1746625709.682037    9302 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


525/756 ━━━━━━━━━━━━━━━━━━━━ 1:38 428ms/step - accuracy: 0.6630 - loss: 0.6563

2025-05-07 09:52:16.142318: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_5837', 4 bytes spill stores, 4 bytes spill loads

2025-05-07 09:52:16.360726: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_5837', 220 bytes spill stores, 220 bytes spill loads

2025-05-07 09:52:16.479920: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_9078', 28 bytes spill stores, 32 bytes spill loads

2025-05-07 09:52:16.487375: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_9078', 20 bytes spill stores, 20 bytes spill loads

2025-05-07 09:52:16.848268: I external/l

756/756 ━━━━━━━━━━━━━━━━━━━━ 0s 432ms/step - accuracy: 0.6668 - loss: 0.6511

2025-05-07 09:53:59.931710: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1702_0', 468 bytes spill stores, 1028 bytes spill loads

2025-05-07 09:54:00.197755: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1702', 488 bytes spill stores, 488 bytes spill loads

2025-05-07 09:54:00.209512: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1702', 80 bytes spill stores, 144 bytes spill loads

2025-05-07 09:54:00.210110: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1702', 608 bytes spill stores, 608 bytes spill loads

2025-05-07 09:54:00.269813: I 

756/756 ━━━━━━━━━━━━━━━━━━━━ 428s 552ms/step - accuracy: 0.6668 - loss: 0.6511 - val_accuracy: 0.6839 - val_loss: 0.6314
Epoch 2/50
756/756 ━━━━━━━━━━━━━━━━━━━━ 377s 499ms/step - accuracy: 0.6821 - loss: 0.6275 - val_accuracy: 0.6839 - val_loss: 0.6221
Epoch 3/50
756/756 ━━━━━━━━━━━━━━━━━━━━ 380s 502ms/step - accuracy: 0.6827 - loss: 0.6153 - val_accuracy: 0.6886 - val_loss: 0.5869
Epoch 4/50
756/756 ━━━━━━━━━━━━━━━━━━━━ 391s 518ms/step - accuracy: 0.6758 - loss: 0.5935 - val_accuracy: 0.6839 - val_loss: 0.5711
Epoch 5/50
756/756 ━━━━━━━━━━━━━━━━━━━━ 388s 513ms/step - accuracy: 0.6842 - loss: 0.5794 - val_accuracy: 0.6867 - val_loss: 0.5517
Epoch 6/50
756/756 ━━━━━━━━━━━━━━━━━━━━ 380s 502ms/step - accuracy: 0.6936 - loss: 0.5683 - val_accuracy: 0.6851 - val_loss: 0.5552
Epoch 7/50
756/756 ━━━━━━━━━━━━━━━━━━━━ 373s 494ms/step - accuracy: 0.6948 - loss: 0.5623 - val_accuracy: 0.7478 - val_loss: 0.5402
Epoch 8/50
756/756 ━━━━━━━━━━━━━━━━━━━━ 372s 492ms/step - accuracy: 0.6972 - loss: 0.55

In [9]:
# model.save("cnn_chest_pretune_resnet.keras")
from tensorflow.keras.models import load_model
model = load_model("cnn_chest_pretune_resnet.keras")
# base_model.trainable = True

In [10]:
classifier_head_weights = [layer.get_weights() for layer in model.layers[-8:]]


In [12]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    #metrics=[AUC(multi_label=True)]
	metrics=['accuracy']
)

In [13]:

for i, layer in enumerate(model.layers[-8:]):
    layer.set_weights(classifier_head_weights[i])

In [23]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    callbacks=[early_stop, reduce_lr]
)

Epoch 1/50


2025-05-07 12:10:09.962113: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_17501_0', 204 bytes spill stores, 420 bytes spill loads

2025-05-07 12:10:10.078705: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_17501', 84 bytes spill stores, 84 bytes spill loads

2025-05-07 12:10:10.140549: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_17501', 4 bytes spill stores, 4 bytes spill loads

2025-05-07 12:10:20.693199: E external/local_xla/xla/service/slow_operation_alarm.cc:73] Trying algorithm eng34{k2=0,k12=-1,k13=2,k14=3,k15=0,k17=128,k18=1,k23=0} for conv %cudnn-conv-bw-input.34 = (f32[32,512,28,28]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,256,14,14]{3,2,1,0}

558/756 ━━━━━━━━━━━━━━━━━━━━ 1:23 424ms/step - accuracy: 0.7153 - loss: 0.9195

2025-05-07 12:14:34.001522: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_17501', 4 bytes spill stores, 4 bytes spill loads

2025-05-07 12:14:34.099187: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_17501_0', 204 bytes spill stores, 444 bytes spill loads

2025-05-07 12:14:34.160879: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_17501', 8 bytes spill stores, 8 bytes spill loads

2025-05-07 12:14:46.194922: E external/local_xla/xla/service/slow_operation_alarm.cc:73] Trying algorithm eng0{} for conv %cudnn-conv-bw-filter.56 = (f32[256,64,1,1]{3,2,1,0}, u8[0]{0}) custom-call(f32[21,64,56,56]{3,2,1,0} %bitcast.65377, f32[21,256,56,56]{3,2,1,0} %bitcast.6

756/756 ━━━━━━━━━━━━━━━━━━━━ 460s 546ms/step - accuracy: 0.7230 - loss: 0.8464 - val_accuracy: 0.4305 - val_loss: 1.4420 - learning_rate: 1.0000e-05
Epoch 2/50
756/756 ━━━━━━━━━━━━━━━━━━━━ 372s 493ms/step - accuracy: 0.8148 - loss: 0.3984 - val_accuracy: 0.7668 - val_loss: 0.4831 - learning_rate: 1.0000e-05
Epoch 3/50
756/756 ━━━━━━━━━━━━━━━━━━━━ 373s 493ms/step - accuracy: 0.8663 - loss: 0.2972 - val_accuracy: 0.8028 - val_loss: 0.4659 - learning_rate: 1.0000e-05
Epoch 4/50
756/756 ━━━━━━━━━━━━━━━━━━━━ 374s 495ms/step - accuracy: 0.9352 - loss: 0.1651 - val_accuracy: 0.8141 - val_loss: 0.5409 - learning_rate: 1.0000e-05
Epoch 5/50
756/756 ━━━━━━━━━━━━━━━━━━━━ 375s 495ms/step - accuracy: 0.9798 - loss: 0.0618 - val_accuracy: 0.8338 - val_loss: 0.6496 - learning_rate: 1.0000e-05
Epoch 6/50
751/756 ━━━━━━━━━━━━━━━━━━━━ 2s 409ms/step - accuracy: 0.9886 - loss: 0.0340
Epoch 6: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.
756/756 ━━━━━━━━━━━━━━━━━━━━ 375s 496ms/step - 

## Stage 2 Bounding Boxes


### Some data pre-processing -> only include target==1 since target==0 has no dimensional values

In [16]:
bounding_boxes_df = labels_df[(labels_df["Target"] == 1) & labels_df["x"].notna()].copy()

bounding_boxes_df.head()

,patientId,x,y,width,height,Target
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1
5,00436515-870c-4b36-a041-de91049b9ab4,562.0,152.0,256.0,453.0,1
8,00704310-78a8-4b38-8475-49f4573b2dbb,323.0,577.0,160.0,104.0,1
9,00704310-78a8-4b38-8475-49f4573b2dbb,695.0,575.0,162.0,137.0,1
14,00aecb01-a116-45a2-956c-08d2fa55433f,288.0,322.0,94.0,135.0,1


In [19]:
bounding_boxes_df['norm_x'] = bounding_boxes_df['x']/1024
bounding_boxes_df['norm_y'] = bounding_boxes_df['y']/1024
bounding_boxes_df['norm_width_x'] = (bounding_boxes_df['x'] + bounding_boxes_df['width'])/1024
bounding_boxes_df['norm_height_y'] = (bounding_boxes_df['y'] + bounding_boxes_df['height'])/1024


In [20]:
bb_train_df, bb_val_df = train_test_split(bounding_boxes_df, test_size=0.2, random_state=42)


In [36]:
# Utilizing this as a source to batch train
# https://www.tensorflow.org/api_docs/python/tf/keras/utils/PyDataset

class DICOMTrainBatchCoordinates(tf.keras.utils.Sequence):
    def __init__(self, df, data_dir, batch_size=32, img_size=(224, 224), shuffle=True):
        self.df = df
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.indices = np.arange(len(df))
        self.on_epoch_end()
    
    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch_df = self.df.iloc[batch_indices]

        batch_images = []
        batch_bb = []

        for _, row in batch_df.iterrows():
            image_path = os.path.join(self.data_dir, f'{row["patientId"]}.dcm')
            image = load_dicom_image(image_path)
            batch_images.append(image)
            batch_bb.append([row['norm_x'], row['norm_y'], row['norm_width_x'], row['norm_height_y']])

        batch_images = np.array(batch_images) / 255.0
        batch_bb = np.array(batch_bb)

        return batch_images, batch_bb

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)


In [37]:
bounding_box_train_batch = DICOMTrainBatchCoordinates(bb_train_df, data_dir)
bounding_box_val_batch = DICOMTrainBatchCoordinates(bb_val_df, data_dir, shuffle=False)

In [40]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout

base_model = MobileNetV2(include_top=False,input_shape=(224,224,3), weights='imagenet')

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)

outputs = Dense(4, activation='sigmoid')(x)

bounding_box_model = tf.keras.Model(inputs=base_model.input,outputs=outputs)
bounding_box_model.compile(optimizer='adam',loss='mean_squared_error', metrics=['accuracy','mae'])

In [41]:
bounding_box_model.fit(bounding_box_train_batch, validation_data=bounding_box_val_batch,epochs=20)

Epoch 1/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 422ms/step - accuracy: 0.5715 - loss: 0.0381 - mae: 0.1606

/home/ojp/projects/tf217/tf217/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2025-05-09 23:02:49.877279: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1207', 4 bytes spill stores, 4 bytes spill loads

2025-05-09 23:02:49.899410: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1207', 4 bytes spill stores, 4 bytes spill loads

2025-05-09 23:03:14.573107: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers a

239/239 ━━━━━━━━━━━━━━━━━━━━ 155s 555ms/step - accuracy: 0.5717 - loss: 0.0380 - mae: 0.1605 - val_accuracy: 0.4097 - val_loss: 0.1233 - val_mae: 0.2963
Epoch 2/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 114s 476ms/step - accuracy: 0.6328 - loss: 0.0249 - mae: 0.1325 - val_accuracy: 0.5081 - val_loss: 0.0674 - val_mae: 0.2059
Epoch 3/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 113s 471ms/step - accuracy: 0.6430 - loss: 0.0238 - mae: 0.1285 - val_accuracy: 0.5924 - val_loss: 0.0670 - val_mae: 0.2200
Epoch 4/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 114s 475ms/step - accuracy: 0.6484 - loss: 0.0231 - mae: 0.1263 - val_accuracy: 0.5777 - val_loss: 0.0412 - val_mae: 0.1720
Epoch 5/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 113s 472ms/step - accuracy: 0.6549 - loss: 0.0228 - mae: 0.1249 - val_accuracy: 0.5908 - val_loss: 0.0439 - val_mae: 0.1794
Epoch 6/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 114s 475ms/step - accuracy: 0.6458 - loss: 0.0241 - mae: 0.1299 - val_accuracy: 0.5924 - val_loss: 0.0545 - val_mae: 0.2001
Epoch 7/20
239/239 ━━━━━━━━

In [42]:
bounding_box_model.save("cnn_chest_pretune_bb.keras")


## Building a 2 stage Pipeline

In [9]:
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pydicom
binary_class_model = load_model("cnn_chest_pretune_resnet.keras")
bounding_box_model = load_model("cnn_chest_pretune_bb.keras")

In [10]:
def prep_image(path):
    img = load_dicom_image(path)
    img = img / 255.0
    return img[np.newaxis,...]

In [11]:
def class_bounding_pipeline(dicom_img, prob_thres=0.4):
    test_img = prep_image(dicom_img)

    prob_class = binary_class_model.predict(test_img)[0][0]
    print(f"Probability of pneumonia: {prob_class:.2f}")

    if prob_class >= prob_thres:
        bb_coor = bounding_box_model.predict(test_img)[0]
        x1,y1,x2,y2 = (bb_coor * 1024).astype(int)
        print("Predicted location of penumonia: ", (x1,y1,x2,y2))
    
        read_img = load_dicom_image(dicom_img)
        plt.imshow(read_img, cmap='gray')
        plt.gca().add_patch(plt.Rectangle((x1, y1), x2 - x1, y2 - y1, edgecolor='red', lw=2, fill=False))
        plt.title(f"Pneumonia Detected (p={prob_class:.2f})")
        plt.axis('off')
        plt.show()

        return prob_class, (x1,y1,x2,y2)
    else:
        print("No pneumonia detected.")
        return prob_class, None




In [14]:
class_bounding_pipeline("./rsna_data/stage_2_test_images/0000a175-0e68-4ca4-b1af-167204a7e0bc.dcm")


I0000 00:00:1746884937.952009    1094 service.cc:152] XLA service 0x7f7548003aa0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1746884937.952042    1094 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2025-05-10 09:48:58.014137: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1746884938.390663    1094 cuda_dnn.cc:529] Loaded cuDNN version 90300


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Probability of pneumonia: 0.26
No pneumonia detected.


I0000 00:00:1746884940.330619    1094 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


(np.float32(0.25548166), None)